# Table of Contents
 <p><div class="lev1"><a href="#How-to-build-a-2D-shovel-crank-system-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to build a 2D shovel-crank system</a></div>

# How to build a 2D shovel-crank system

F. Dubois - 2016

In [ ]:
#Initialization

from pathlib import Path
import numpy as np
from pylmgc90 import pre

datbox = Path('DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

# containers
#   * bodies
bodies = pre.avatars()
#   * materials
mats   = pre.materials()
#   * see tables
svs    = pre.see_tables()
#   * contact laws
tacts  = pre.tact_behavs()

tt=0.


Defining a material/model suitable for rigid objects. 

In [ ]:
mat  = pre.material(name='TDURx', materialType='RIGID', density=1000.)
mats+= mat

mod  = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

Crank:
 motor ...

In [ ]:
disk = pre.avatar(dimension=2)
# new node
no = pre.node(coor=np.array([0.,0.]),number=1)
# adding no to avatar
disk.addNode( no )
# new element 
disk.addBulk( pre.rigid2d() )
#
disk.defineGroups()
#
disk.defineModel(model=mod)
disk.defineMaterial(material=mat)
#
disk.addContactors(shape='DISKx', color='BLUEx', byrd=0.5)
disk.addContactors(shape='PT2Dx', color='VERTx', shift=[0., 0.])
disk.addContactors(shape='PT2Dx', color='VERTx', shift=[0.1,0.])
disk.computeRigidProperties()

disk.imposeDrivenDof(component=[1, 2], dofty='vlocy')
disk.imposeDrivenDof(component=[3], dofty='vlocy',ct=1.,rampi=1.)

bodies+=disk

 ... arm 1 ...

In [ ]:
arm1 = pre.avatar(dimension=2)
# new node
no = pre.node(coor=np.array([1,0.]),number=1)
# adding no to avatar
arm1.addNode( no )
# new element 
arm1.addBulk( pre.rigid2d() )
#
arm1.defineGroups()
#
arm1.defineModel(model=mod)
arm1.defineMaterial(material=mat)
#
arm1.addContactors(shape='JONCx', color='WALLx', axe1=1., axe2=0.1)
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[-1.,0.])
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[-0.9,0.])
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[1.,0.])
arm1.computeRigidProperties()
bodies+=arm1

 ... arm2 ...

In [ ]:
arm2 = pre.avatar(dimension=2)
# new node
no = pre.node(coor=np.array([5.,0.]),number=1)
# adding no to avatar
arm2.addNode( no )
# new element 
arm2.addBulk( pre.rigid2d() )
#
arm2.defineGroups()
#
arm2.defineModel(model=mod)
arm2.defineMaterial(material=mat)
#
arm2.addContactors(shape='JONCx', color='WALLx', axe1=3., axe2=0.1)
arm2.addContactors(shape='PT2Dx', color='VERTx', shift=[-3.,0.])
arm2.addContactors(shape='PT2Dx', color='VERTx', shift=[3.,0.])
arm2.computeRigidProperties()

bodies+=arm2

shovel

In [ ]:
yy=-3.

shovel = pre.avatar(dimension=2)
# new node
no = pre.node(coor=np.array([8.,yy]),number=1)
# adding no to avatar
shovel.addNode( no )
# new element 
shovel.addBulk( pre.rigid2d() )
#
shovel.defineGroups()
#
shovel.defineModel(model=mod)
shovel.defineMaterial(material=mat)
#
shovel.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                     vertices=np.array([[-0.1,-2],[0.1,-2],[0.1,2.],[-0.1,2]]))
shovel.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                     vertices=np.array([[-0.1,2.],[-0.5,2.],[-0.5,1.9],[-0.1,1.9]]))
shovel.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                     vertices=np.array([[0.1,-2],[0.5,-2],[0.5,-1.9],[0.1,-1.9]]))

shovel.addContactors(shape='PT2Dx', color='VERTx', shift=[0.,-yy])
#
shovel.computeRigidProperties()
#
shovel.imposeDrivenDof(component=[1, 2], dofty='vlocy')
#
bodies+=shovel


In [ ]:
down = pre.rigidJonc(axe1=3., axe2=0.1, center=[11.,yy-2.], model=mod, material=mat, color='REDxx')
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
bodies+=down

left = pre.rigidJonc(axe1=2., axe2=0.1, center=[8.,yy], model=mod, material=mat, color='REDxx')
left.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
left.rotate(description='axis', center=left.nodes[1].coor, axis=[0.,0.,1.], 
            alpha=np.pi/2.)
bodies+=left 

right = pre.rigidJonc(axe1=2., axe2=0.1, center=[14.,yy], model=mod, material=mat, color='REDxx')
right.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
right.rotate(description='axis', center=right.nodes[1].coor, axis=[0.,0.,1.], 
             alpha=np.pi/2.)
bodies+=right 


In [ ]:
nb_particles = 100
radii = pre.granulo_Random(nb_particles, 0.1, 0.3)

# depot dans une boite rectangulaire
lx = 5.8
ly = 4. 
nb_laid_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)

# si toutes les particules deposees n'ont pas ete conservees
if (nb_laid_particles < nb_particles):
   # on affiche un avertissement
   print("Warning: granulometry changed, since some particles were removed!")

# boucle d'ajout des disques :
for r, c in zip(radii, coor):
    # creation un nouveau disque rigide, constitue du materiau plex
    body = pre.rigidDisk(r=r, center=c, model=mod, material=mat, color='REDxx') 
    body.translate(dx=8.1,dy=yy-1.9) 
    # ajout du disque dans le conteneur de corps
    bodies += body

contact laws

In [ ]:
lcpl  = pre.tact_behav(name='xxxxx', law='COUPLED_DOF')
tacts+= lcpl
liqs  = pre.tact_behav(name='iqsc0', law='IQS_CLB', fric=0.3)
tacts+= liqs

sv1 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='PT2Dx', colorCandidat   ='VERTx',
                    behav=lcpl, 
                    CorpsAntagoniste='RBDY2', antagoniste='PT2Dx', colorAntagoniste='VERTx',
                    alert=0.05)
svs+=sv1

sv2 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='POLYG', colorCandidat   ='REDxx',
                    behav=liqs, 
                    CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='REDxx',
                    alert=0.05)
svs+=sv2

sv3 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='REDxx',
                    behav=liqs, 
                    CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='REDxx',
                    alert=0.05)
svs+=sv3

sv4 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='REDxx',
                    behav=liqs, 
                    CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='REDxx',
                    alert=0.05)
svs+=sv4


In [ ]:
# writing files

post = pre.postpro_commands()
my_command = pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)
#

pre.writeDatbox(dim, mats, [mod], bodies, tacts, svs, post=post, datbox_path=datbox)

try:
    pre.visuAvatars(bodies)
except:
    pass

In [ ]:

#  disk.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
#                     vertices=np.array([[-1.,-1.],[1.,-1.],[1.,1.],[-1.,1.]]))
